<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing II</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=463">MICRO-512</a>) taught by Dr. D. Sage, Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2024.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:sebastien.herbreteau@epfl.ch">Sébastien Herbreteau</a> and <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
     
</p>
<hr style="clear:both">
<h1>Lab 7.2:  Introduction to Pyxu </h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: Thursday May 23, 2024</p>
    <p style="margin:4px;"><b>Submission</b>: <span style="color:red">Monday June 3, 2024</span> (before 11:59PM) on <a href="https://moodle.epfl.ch/course/view.php?id=463">Moodle</a></p>
    <p style="margin:4px;"><b>Total number of points</b>: 21</p>
    <p style="margin:4px;"><b>Helps session</b>: 10:15 - 12:00, Thursday May 30, 2024 in CM 2</p>     
    <p style="margin:4px;"><b>Related lectures</b>: Chapters 9 and 10</p>
</div>

Double-click on this cell, fill your name and SCIPER number below to verify your identity in Noto and set the seed for random results.
:::{attention} Please write down your name and SCIPER! 
### Student Name: 
### SCIPER: 
:::

In [ ]:
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

## <a name="imports_"></a> Imports
In the next cell we import Python libraries we will use throughout the lab.

In [ ]:
# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import skimage
from interactive_kit import imviewer as viewer

import pyxu
from pyxu.operator import Convolve, DiagonalOp, PositiveL1Norm, SquaredL2Norm, L1Norm, Gradient, PositiveOrthant
from pyxu.opt.solver import PGD
from pyxu.operator.interop import from_source
from pyxu.opt.stop import MaxIter

# Configure plotting as dynamic
%matplotlib widget

# Introduction to Pyxu (7 points)

## <a id="ToC_2_NN"></a>Table of contents
1. [Pyxu in a Nutshell](#1.-Pyxu-in-a-Nutshell) **(0 point)**
2. [Hands-on Pyxu with TV regularization](#2.-Hands-on-Pyxu-with-TV-regularization) **(3 points)**
3. [Application: tomographic reconstruction](#3.-Application:-tomographic-reconstruction) **(4 points)**

## 1. Pyxu in a Nutshell
[Back to table of contents](#ToC_2_NN)

<a href="https://pyxu-org.github.io/">Pyxu</a> is an open-source computational imaging software framework for Python 3, developed at EPFL. It is based on **operators**. When we talk about a **forward operator** in computational imaging or signal processing, we are describing a transformation that takes an input (often some form of data or signal) and produces an output (often some form of measurement or transformed data). Complex  **operators** can be constructed by composing Pyxu's fundamental building blocks via the following set of arithmetic operations:
```python
>> op1 + op2 # Addition of two operators
>> op1 * op2 # Composition of two operators
>> op ** 3   # Exponentiation of an operator
>> 4 * op # Scaling
```

#### Defining operators for the forward model

In the case of inverse problems in imaging, **operators** are used to define the forward model. More precisely, consider the situation where we have at our disposal a measurement $y$ which is the realization of a forward model $\Phi(s)$ where $s$ is to be determined. The function $\Phi$ is described in Pyxu via **forward operators**. For example, if $\Phi$ describes a convolution followed by partial masking, we would write for example:

In [ ]:
kernel = skimage.io.imread('images/psf_movement.tif').astype(np.float64)
# Operator convolution for a 512x512 image
Conv = Convolve(arg_shape=(512, 512), kernel=kernel, center=(kernel.shape[0]//2, kernel.shape[1]//2), mode="reflect")

np.random.seed(1234) # for reproductibility, do not modify
mask = np.random.binomial(n=1, p=0.95, size=(512, 512))
# Operator of partial masking for a 512x512 image
Mask = DiagonalOp(mask.ravel()) # operators only accept vectorized inputs

# Compose operators
Phi = Mask * Conv

Thus defined, $\Phi$ is an **operator** which can be applied to any image of size $512 \times 512$, in particular the image from the previous notebook.

<div class="alert alert-danger">
    <b>Beware</b>: in the world of Pyxu, operators accept only vectorized inputs (method <code>.ravel()</code> in NumPy).
</div>

In [ ]:
einstein = skimage.io.imread('images/einstein.tif').astype(np.float64)
masked_and_blurred_einstein = Phi(einstein.ravel()).reshape(einstein.shape)
np.random.seed(1234) # for reproductibility, do not modify
masked_and_blurred_and_noisy_einstein = masked_and_blurred_einstein + 10 * np.random.randn(*masked_and_blurred_einstein.shape)

plt.close('all')
view = viewer([masked_and_blurred_einstein, masked_and_blurred_and_noisy_einstein, einstein, kernel], widgets=True, hist=False, axis=True, cmap='gray')

#### Defining operators for the data-fidelity term

The data-fidelity term is associated with the forward model. Here data-fidelity is expressed in terms of a squared $\ell_2$ norm.

In [ ]:
np.random.seed(1234) # for reproductibility, do not modify
y = masked_and_blurred_and_noisy_einstein
sl2 = SquaredL2Norm(dim=y.size) # defines the loss \| . \|_2^2
sl2 = sl2.asloss(y.ravel()) # defines the loss \| . - y \|_2^2
F = sl2 * Phi # defines the data-fidelity loss \| \Phi(.) - y \|_2^2

#### Defining operators for the regularization terms

To deal with ill-posed problems, it is common practice to add regularization terms to the optimization problem to be solved, which favor certain behaviors of the solution. Once again, regularization terms are also defined by **operators** in Pyxu. For example, if one wants to define a $\ell_1$ norm penalization to promote sparsity with hyperparameter $\lambda$, one would write:

In [ ]:
lam = 1e2
G = lam * L1Norm(dim=y.size)

#### Solvers

Pyxu offers multiple solvers for which an exhaustive list can be found <a href="https://pyxu-org.github.io/api/opt.solver.html">here</a>. Among them is the <a href="https://pyxu-org.github.io/api/opt.solver.html#pyxu.opt.solver.PGD">**Proximal Gradient Descent (PGD)**</a> solver which was introduced in the previous notebook. Note that Pyxu automatically computes upper bounds of the maximum eigenvalue 
 of composite operators, which is useful for automatically choosing suitable step sizes in optimisation algorithms (done under the hood by Pyxu's algorithmic suite).
 
<div class="alert alert-info">
<b>Note:</b> As mentionned in the <a href="https://pyxu-org.github.io/api/opt.solver.html#pyxu.opt.solver.PGD">documentation</a>, for the solver <code>PGD</code>, $f$ should be convex and differentiable with $L$-Lipschitz continuous gradient and $g$ should be convex, not necessarily differentiable, with a simple proximal operator. This is the case here since the proximal operator of the $\ell_1$ norm can be computed exactly with the soft threshold. 
</div>

In [ ]:
 # Define the solver
solver = PGD(f=F, g=G, show_progress=False) 

# Solve the optimization problem
solver.fit(x0=np.zeros(y.ravel().shape), stop_crit=MaxIter(50)) # initialization: all-zeros vector; number of iterations fixed to 100

# Get the solution and reshape to the original size
s_opt = solver.solution().reshape(y.shape)

In [ ]:
plt.close('all')
view = viewer([s_opt, y], widgets=True, hist=False, axis=True, cmap='gray')

<div class="alert alert-info">
    <b>Note:</b> Unsurprisingly, the resulting image is of poor quality as promoting sparsity is not appropriate for this kind of image.
</div>

## 2. Hands-on Pyxu with TV regularization
[Back to table of contents](#ToC_2_NN)

For **2 points**, using Pyxu and the **Proximal Gradient Descent** algorithm, compute the solution to the following optimization problem **under positivity constraints**:
$$s_\lambda^\ast = \arg \min_{s \geq \mathbf{0}} \| \Phi(s) - y \|_2^2 + \lambda \| \nabla s \|_2^2$$
where $\lambda=0.05$, $\Phi$ is the function defined in the previous section (convolution with psf of movement followed by partial masking) and $y$ is the blurred, masked and noisy Einstein. Pay attention that, contrary to the previous notebook, the squared $\ell_2$ norm was chosen to ensure differentiability.

<div class="alert alert-danger">
    <b>Beware:</b> For reasons of time, stop the algorithm after 50 iterations and take the $y$ vector as initialization (the algorithm should finish in less than 45 seconds).
</div>

<div class="alert alert-info">
    <b>Hints:</b> 
    
1. You may want to use the operators <a href="https://pyxu-org.github.io/api/operator/func.html#pyxu.operator.PositiveOrthant">PositiveOrthant</a> and <a href="https://pyxu-org.github.io/api/operator/linop.html#pyxu.operator.Gradient">Gradient</a> with directions equal to (0, 1).

2. Be careful with the output dimension of the Gradient operator. 

3. Be careful with the terms dedicated to the `f` argument of the solver `PGD` and those dedicated to the `g` argument.
</div>

In [ ]:
lam = 5e-2 # Regularization parameter

# Define the operators
# YOUR CODE HERE

# Define and fit the solver
# YOUR CODE HERE

# Get solution
s_opt = None
# YOUR CODE HERE


# Define and fit the solver
plt.close('all')
view = viewer([s_opt, y], widgets=True, hist=False, axis=True, cmap='gray')

**For 1 point**, answer the following MCQ:
* What is the SNR of the solution found with Pyxu ? (**1 point**)
1. $20.08$ dB
2. $21.08$ dB
3. $22.08$ dB
4. $23.08$ dB
5. $24.08$ dB
6. $25.08$ dB
7. $26.08$ dB 

<div class="alert alert-info">
<b>Hint:</b> You may use the next cell to use or redefine the SNR function that you implemented in the first notebook.
</div>

In [ ]:
# YOUR CODE HERE

Modify the variable `answer` in the next cell to reflect your choices.

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

## 3. Application: tomographic reconstruction
[Back to table of contents](#ToC_2_NN)

Tomographic reconstruction is a computational process used in imaging sciences to reconstruct 2D or 3D images from a series of projections captured at different angles. 

The fundamental principle behind tomographic reconstruction is the Radon transform, mapping a function in 2D or 3D space to a set of line integrals. In the context of medical imaging, these line integrals represent the attenuation of X-rays as they pass through a human body. By capturing these attenuations from various angles, it is possible to reconstruct a cross-sectional image of the internal structure.

The process typically involves the following steps:

- Data Acquisition: Projection data is gathered from different angles, typically by rotating the X-ray source and detector around the object being imaged. The acquired projection data are then ‘smeared’ back across the imaging field for each angle (back-projection). The data representation that stacks 1D projections of an object at various angles is called a sinogram.

- Reconstruction: Various algorithms, such as Filtered Back-Projection (FBP) or iterative methods, are used to compute the original image from the smeared data. However, due to the non-idealities inherent to the tomographic setup, regularization is often useful to reconstruct the original object.

### Generating ideal sinogram (simulation of real data acquisition)

Given an image (for example `images/MRI.png`), we can simulate the real data acquisition process using the Radon transform (the Pyxu operators `Radon` and its inverse `InvRadon` are given below). In this example, projection data is gathered from 360 angles around the brain. The result is an ideal sinogram.

In [ ]:
# Import brain image
phantom = skimage.io.imread('images/MRI.png').astype(np.float64)[::4, ::4] / 255 # downsampling for speed issues (128x128 image)
nb_angles = 360

In [ ]:
# Radon Operator 
Radon = from_source(cls=pyxu.abc.LinOp,
                    shape=(phantom.shape[0]*nb_angles, phantom.size),
                    apply=lambda _, arr: skimage.transform.radon(arr.reshape(phantom.shape),
                                                   theta=np.linspace(0, 180, nb_angles),
                                                   circle=True).ravel(),
                    adjoint=lambda _, arr: skimage.transform.iradon(arr.reshape(phantom.shape[0], nb_angles),
                                                      filter_name=None,
                                                      circle=True).ravel(),
                    vectorize=["apply", "adjoint"],
                    vmethod="scan",
                    enforce_precision=["apply", "adjoint"])

# Inverse Radon Operator
InvRadon = from_source(cls=pyxu.abc.LinOp,
                    shape=(phantom.size, phantom.shape[0]*nb_angles),
                    apply=lambda _, arr: skimage.transform.iradon(arr.reshape(phantom.shape[0], nb_angles),
                                                      filter_name="hamming",
                                                      circle=True).ravel())

In [ ]:
# Compute ideal sinogram
ideal_sino = Radon(phantom.ravel()).reshape(phantom.shape[0], nb_angles)

plt.close('all')
view = viewer([phantom, ideal_sino], widgets=True, hist=False, axis=True, cmap='gray')

### Reconstruction with Filtered Back-Projection (FBP)

In the ideal case, the reconstructed object can be well recovered from its sinogram using the algorithm of Filtered Back-Projection, implemented through the given Pyxu operator `InvRadon`.

In [ ]:
ideal_fbp = InvRadon(ideal_sino.ravel()).reshape(phantom.shape)

plt.close('all')
view = viewer([ideal_fbp, phantom, ideal_fbp - phantom], title=["Ideal FBP", "Phantom", "Difference"], widgets=True, hist=False, axis=True, cmap='gray')

### Reconstruction from a non-ideal sinogram

However, in practice, the process of acquisition deviates from the assumptions usually made by traditional Radon transform models. The complexities of the studied setup in this example include:

- Detector Width: Detectors feature non-negligible widths, hence perform tube integrals in place of line integrals assumed by the Radon transform. This results in a blurring effect in the sinogram.

- Probabilistic Element: There’s a chance that detector pixels might malfunction during each scan, introducing missing values in the sinogram.

- X-ray Beam Intensity: The X-ray beam’s intensity tapers towards the edges, causing a similar tapering effect in the sinogram, which complicates the application of the standard Radon transform.

Each of the perturbations are described with Pyxu by the following operators:

In [ ]:
# Detector Width
kernel = np.ones((3, 1)); kernel /= kernel.sum()
Blur1D = Convolve(arg_shape=ideal_sino.shape, kernel=kernel, center=(kernel.shape[0]//2, kernel.shape[1]//2), mode="reflect")

# Probabilisic Element
np.random.seed(1234) # for reproductibility, do not modify
mask = np.random.binomial(n=1, p=0.99, size=ideal_sino.shape)
Mask = DiagonalOp(mask.ravel())

# X-ray beam intensity
taper = np.hamming(ideal_sino.shape[0]).reshape(-1, 1).repeat(ideal_sino.shape[1], axis=1)
BeamIntensity = DiagonalOp(taper.ravel())

# Compose all operators
Phi = BeamIntensity * Mask * Blur1D * Radon

In [ ]:
non_ideal_sino = Phi(phantom.ravel()).reshape(ideal_sino.shape)

plt.close('all')
view = viewer([non_ideal_sino, ideal_sino], widgets=True, hist=False, axis=True, cmap='gray')

#### Naive application of Filtered-Back Projection

**For 1 point** answer the following MCQ:
* What is the SNR of naive application of Filtered-Back Projection to the non-ideal sinogram ? (**1 point**)
1. $1.15$ dB
2. $2.15$ dB
3. $3.15$ dB
4. $4.15$ dB
5. $5.15$ dB
6. $6.15$ dB
7. $7.15$ dB

Use the next cell to compute it.

In [ ]:
non_ideal_fbp = None
# YOUR CODE HERE

plt.close('all')
view = viewer([non_ideal_fbp, phantom, non_ideal_fbp - phantom], title=["Non-ideal FBP", "Phantom", "Difference"], widgets=True, hist=False, axis=True, cmap='gray')

Modify the variable `answer` in the next cell to reflect your choices. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

#### Leveraging Pyxu for regularization

For **2 points**, solve the reconstruction problem **under positive constraints**:
$$s^\ast = \arg \min_{s \geq 0} \| \Phi(s) - y \|_2^2 + \lambda  \| s \|_1 + \mu \| \nabla s \|_2^2$$
with $\lambda=1$ and $\mu=10^{-4}$, and where $y$ represents the non-ideal sinogram and $\Phi$ is the composition of the radon transform followed by all the three perturbations defined previously. To do so, use the **Proximal Gradient Descent** solver from Pyxu with gradient step size equal to $10^{-3}$.

<div class="alert alert-danger">
<b>Beware:</b> For reasons of time, stop the algorithm after 200 iterations and take the all-zeros vector as initialization (<b>takes about 3 minutes</b>).
</div>

<div class="alert alert-info">
<b>Hints:</b> 
    
1. You may want to use the operator <a href="https://pyxu-org.github.io/api/operator/func.html#pyxu.operator.PositiveOrthant">PositiveL1Norm</a>.

2. The gradient step size is defined with argument `tau` in the method `fit`.
</div>

In [ ]:
y = non_ideal_sino
lam = 1.0 # Regularization parameter
mu = 1e-4 # Regularization parameter

# Define the operators
# YOUR CODE HERE


# Define and fit the solver with all-zeros vector as initialization
# YOUR CODE HERE


# Get solution
s_opt = None
# YOUR CODE HERE

# Display the solution and observe the differences between all the estimators of the phantom image
plt.close('all')
view = viewer([s_opt, non_ideal_fbp, ideal_fbp, phantom], title=["Pyxu solution", "Non-ideal FBP", "Ideal FBP", "Phantom"], widgets=True, hist=False, axis=True, cmap='gray')

For **1 point** answer the following MCQ:
* What is the SNR of the reconstructed image ? (**1 point**)
1. $1.76$ dB
2. $2.76$ dB
3. $3.76$ dB
4. $4.76$ dB
5. $5.76$ dB
6. $6.76$ dB
7. $7.76$ dB

Use the next cell to compute it. 

In [ ]:
# YOUR CODE HERE

Modify the variable `answer` in the next cell to reflect your choices.

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')